In [8]:
import random
from itertools import product

# 定义变量
defective_rate = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
purchase_costs = [2, 8, 12, 2, 8, 12, 8, 12]
inspection_costs = [1, 1, 2, 1, 1, 2, 1, 2]

assembly_cost = 8
assembly_inspection_cost = 4
disassembly_semi_cost = 6
disassembly_final_cost = 10
market_loss = 40

# 随机决定是否为次品
def random_defective(rate):
    return random.random() < rate

# 零配件检测
def inspect_part(part_index, inspect):
    total_cost = 0
    while True:
        cost = purchase_costs[part_index]
        total_cost += cost
        if inspect:
            total_cost += inspection_costs[part_index]
            if random_defective(defective_rate[part_index]):
                continue  # 零件不合格，重新购买并检测
        else:
            # 如果不检测，也根据次品率决定是否合格
            if random_defective(defective_rate[part_index]):
                return total_cost, False  # 不合格，不检测时发现次品

        # 只在检测合格或不检测且零件通过时返回
        return total_cost, True  # 合格


# 半成品检测
import random

def assemble_semi_product(parts_indices, inspect_parts, inspect_semi, discard_if_defective, max_retries=3):
    total_cost = 0
    all_parts_pass = True
    retries = 0

    # 半成品不合格的概率为10%
    def semi_product_defective():
        return random.random() < 0.1

    while retries <= max_retries:
        all_parts_pass = True
        # 检测每个零件
        for part_index in parts_indices:
            cost, part_pass = inspect_part(part_index, inspect_parts[part_index])
            total_cost += cost
            all_parts_pass = all_parts_pass and part_pass

        # 装配成本
        total_cost += assembly_cost

        # 检测半成品
        if inspect_semi:
            total_cost += assembly_inspection_cost
            if semi_product_defective():
                if discard_if_defective:
                    # 丢弃半成品并重新装配
                    total_cost += assembly_cost  # 新装配半成品的成本
                    retries += 1
                    continue  # 重新装配并检测
                else:
                    # 拆解半成品并重新装配
                    total_cost += disassembly_semi_cost  # 拆解半成品的成本
                    retries += 1
                    continue  # 重新装配并检测
        break  # 合格或者不检测时跳出循环

    return total_cost, all_parts_pass


# 成品检测
def assemble_final_product(inspect_parts, inspect_semi_products, inspect_final, discard_if_defective):
    total_cost = 0
    all_semi_pass = True

    # 半成品的零件集合
    semi_products = [[0, 1, 2], [3, 4, 5], [6, 7]]

    # 检测每个半成品
    for semi_index, semi_product in enumerate(semi_products):
        inspect_semi = inspect_semi_products[semi_index]  # 对应的半成品是否检测
        cost, semi_pass = assemble_semi_product(semi_product, inspect_parts, inspect_semi, discard_if_defective)
        total_cost += cost
        all_semi_pass = all_semi_pass and semi_pass

    # 装配成品成本
    total_cost += assembly_cost

    # 检测成品
    if inspect_final:
        total_cost += assembly_inspection_cost
        if not all_semi_pass:
            total_cost += disassembly_final_cost  # 拆解不合格成品
            return total_cost, False
    return total_cost, all_semi_pass

# 模拟退货处理
def handle_returned_product(inspect_parts, inspect_semi_products, inspect_final, discard_if_defective):
    total_cost, product_pass = assemble_final_product(
        inspect_parts=inspect_parts, 
        inspect_semi_products=inspect_semi_products, 
        inspect_final=inspect_final,
        discard_if_defective=discard_if_defective
    )

    # 若不合格且未检测出，进入市场
    if not product_pass:
        total_cost += market_loss
    return total_cost

# 策略模拟函数
def simulate_strategy(inspect_parts, inspect_semi_products, inspect_final, discard_if_defective, num_simulations=1000):
    total_cost = 0
    for _ in range(num_simulations):
        # 模拟成品装配与检测过程
        cost, product_pass = assemble_final_product(
            inspect_parts=inspect_parts, 
            inspect_semi_products=inspect_semi_products, 
            inspect_final=inspect_final,
            discard_if_defective=discard_if_defective
        )
        total_cost += cost

        # 若不合格成品进入市场
        if not product_pass:
            total_cost += market_loss

    # 计算平均成本
    return total_cost / num_simulations

# 遍历所有零配件、半成品和成品检测组合
def find_optimal_strategy():
    best_cost = float('inf')
    best_strategy = None

    # 遍历零件的检测情况（2^8种组合）
    for inspect_parts in product([True, False], repeat=8):
        # 遍历半成品的检测情况（2^3种组合）
        for inspect_semi_products in product([True, False], repeat=3):
            # 遍历成品的检测情况（2种组合）
            for inspect_final in [True, False]:
                # 遍历丢弃和拆解策略（2种组合）
                for discard_if_defective in [True, False]:
                    # 模拟策略
                    cost = simulate_strategy(
                        inspect_parts=inspect_parts, 
                        inspect_semi_products=inspect_semi_products, 
                        inspect_final=inspect_final,
                        discard_if_defective=discard_if_defective
                    )

                    # 打印每种策略的结果
                    print(f"零配件检测策略: {inspect_parts}, 半成品检测策略: {inspect_semi_products}, 成品检测: {inspect_final}, 丢弃不合格半成品: {discard_if_defective}, 平均成本: {cost}")

                    # 比较成本，找到最优策略
                    if cost < best_cost:
                        best_cost = cost
                        best_strategy = (inspect_parts, inspect_semi_products, inspect_final, discard_if_defective)

    return best_strategy, best_cost

# 找到最优检测策略
best_strategy, best_cost = find_optimal_strategy()

print("最优检测策略:", best_strategy)
print("最低平均成本:", best_cost)



零配件检测策略: (True, True, True, True, True, True, True, True), 半成品检测策略: (True, True, True), 成品检测: True, 丢弃不合格半成品: True, 平均成本: 149.79
零配件检测策略: (True, True, True, True, True, True, True, True), 半成品检测策略: (True, True, True), 成品检测: True, 丢弃不合格半成品: False, 平均成本: 146.759
零配件检测策略: (True, True, True, True, True, True, True, True), 半成品检测策略: (True, True, True), 成品检测: False, 丢弃不合格半成品: True, 平均成本: 145.443
零配件检测策略: (True, True, True, True, True, True, True, True), 半成品检测策略: (True, True, True), 成品检测: False, 丢弃不合格半成品: False, 平均成本: 142.821
零配件检测策略: (True, True, True, True, True, True, True, True), 半成品检测策略: (True, True, False), 成品检测: True, 丢弃不合格半成品: True, 平均成本: 138.333
零配件检测策略: (True, True, True, True, True, True, True, True), 半成品检测策略: (True, True, False), 成品检测: True, 丢弃不合格半成品: False, 平均成本: 136.85
零配件检测策略: (True, True, True, True, True, True, True, True), 半成品检测策略: (True, True, False), 成品检测: False, 丢弃不合格半成品: True, 平均成本: 133.419
零配件检测策略: (True, True, True, True, True, True, True, True), 半成品检测策略: (True, True, Fa